<a href="https://colab.research.google.com/github/lesya7maltseva/pure-Python-/blob/main/%22%D0%9A%D0%B5%D0%B9%D1%81_%D0%9B%D0%BE%D0%B3%D0%B8%22_list%2Cdict_comprehansions%2C_%D1%80%D0%B5%D0%B3%D1%83%D0%BB%D1%8F%D1%80%D0%BA%D0%B8_findall%2CCounter%2Csorted.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **1 кейс**

**Работа с логами**

**Важно**

Перед началом решения выполните следующую ячейку, чтобы загрузить необходимый для работы файл.

In [2]:
!wget https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log

--2025-02-26 04:23:59--  https://gist.github.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.github.com (gist.github.com)... 140.82.113.3
Connecting to gist.github.com (gist.github.com)|140.82.113.3|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log [following]
--2025-02-26 04:23:59--  https://gist.githubusercontent.com/Vs8th/38d5d914171c84166728a9746d212bad/raw/auto_purchase.log
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.108.133, 185.199.110.133, 185.199.111.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 459418 (449K) [text/plain]
Saving to: ‘auto_purchase.log’

auto_purchase.log   100%[===================>] 448.65K  --.-KB/s    in 0.04s   

2025-02-26 04:24:00 (9.97

Чтобы посмотреть как он выглядит выполните следующую ячейку.

In [ ]:
with open('auto_purchase.log', 'r') as f:
    lines = f.readlines()

for line in lines[400:550]:
    print(line)

### **Решения**

#### **Задача 1**

Ваша задача написать функцию `count_success_and_failure`, которая принимает на вход путь к файлу с логами и подсчитывает количество успешных продлений и ошибок при списании. Функция должна вернуть кортеж из двух значений: количества успешных попыток и неуспешных.

**Решение**

Напишите свое решение ниже

In [4]:
import re
from collections import Counter
from statistics import median
import datetime
from datetime import datetime

In [ ]:

def count_success_and_failure(file_path):
    with open(file_path, 'r') as f:
      lines = f.readlines()
    lst_ = [re.findall(r'Обновляем подписку|ошибка при списании', l)  for l in lines ]
    lst_2 = [k for el in lst_ for k in el]
    dict_payments = Counter(lst_2)
    dict_payments['Обновляем подписку'] = dict_payments['Обновляем подписку'] - dict_payments.get('ошибка при списании')
    return tuple(dict_payments.values())
    ...

count_success_and_failure('auto_purchase.log')

(1034, 186)

#### **Задача 2**

Ваша задача написать функцию `auto_renewal_sub`, которая принимает на вход путь к файлу с логами и обрабатывает количество клиентов с автопродлением подписки. Мы хотим посмотреть на изменение этого показателя в динамике: посчитайте сглаженные значения с помощью метода скользящего среднего и метода медианного сглаживания.  

**Примечание:** При сглаживании берем все предыдущие значения, включая текущее, будущие значения не берем. Если в один день наблюдаем несколько записей об автопродлении - берем максимальное из имеющихся число клиентов с подпиской.

Функция должна записать в файл `auto_renewal_sub.txt` два списка, предварив их соответствущими обозначениями:

`Среднее: [2.0, 1.0, 0.67...]`

`Медиана: [2, 2, 0...]`

**Решение**

Напишите свое решение ниже

In [5]:
def auto_renewal_sub(log_file_path):
    with open(log_file_path) as f:
      lines = f.readlines()

    # собираем словарь, вытаскивая регулярками дату и сколько было пользователей
    dict_raw ={}
    for l in lines:
      key = re.findall(r'\d{4}-\d{2}-\d{2},',l)
      if key:
        val = re.findall(r'\d{1}\n|\d{2}\n',l)
        if key[0][:-1] in dict_raw.keys():
          dict_raw[key[0][:-1]].append(int(val[0].strip()))
        else: dict_raw[key[0][:-1]] = [int(val[0].strip())]

    # переделываем ключи из даты в числа для следующей логики
    dict_avg = {key: [max(val)] for key, val in zip(range(1, len(dict_raw)+1) ,dict_raw.values())}

    #собираем для подсчета накопительный итог:
    dict_cumulative = {}
    for key, val in dict_avg.items():
      if key>1:
        dict_cumulative[key] = val + dict_cumulative[key-1]
      else: dict_cumulative[key] = val

    #список со средним арифметическим
    lst_avg = [round(sum(el)/len(el),2) for el in dict_cumulative.values()]

    #список с медианой
    lst_avg_med = [int(median(el)) for el in dict_cumulative.values()]

    #Пишем в файл:
    with open('auto_renewal_sub.txt', 'w') as f:
      f.write(f"Среднее: {lst_avg}\n")
      f.write(f"Медиана: {lst_avg_med}")

    ...

auto_renewal_sub('auto_purchase.log')

#### **Задача 3**

Напишите функцию `sub_renewal_by_day`, которая принимает на вход путь к файлу с логами и анализирует взаимосвязь дня продления подписки и количества продлений в этот день. Функция должна записать в файл `weekdays.txt` аналитическую записку в формате:

**`Количество обновлений подписки по дням недели:`**

**`Понедельник: 6`**

**`Вторник: 7`**

**`Среда: 8`**

**`...`**

**Решение**

Напишите свое решение ниже

In [ ]:

def sub_renewal_by_day(file_path):
    with open(file_path, 'r') as f:
      lst_lines = f.readlines()

    # убираем тех, кто не оплатил
    for i in range(len(lst_lines)):
      if i < len(lst_lines) - 1 :
        if lst_lines[i+1].startswith('ERROR'): lst_lines.pop(i)
    lst_dates = []

    # выбираем только логи оплаты и собираем даты в новый список
    for line in lst_lines:
      str_check = re.findall(r'\d{4}-\d{2}-\d{2}[\s\S]*payment_method_id:',line)
      if str_check: lst_dates.append(re.findall(r'\d{4}-\d{2}-\d{2}',line)[0])

    # Переделываем даты в номер дня
    lst_days = [datetime.strptime(dt, '%Y-%m-%d').weekday() for dt in lst_dates]

    #Считаем сколько в какой день подписок. Уииии Counter
    dict_count = Counter(lst_days)
    dict_count = dict(sorted(dict_count.items()))
    dict_days = {0: 'Понедельник', 1:'Вторник', 2: 'Среда', 3: 'Четверг', 4: 'Пятница', 5: 'Суббота', 6: 'Воскресенье' }

    with open('weekdays.txt', 'w') as f:
      f.write('Количество обновлений подписки по дням недели:')
      for key, val in dict_count.items():
        f.write('\n')
        f.write(f"{dict_days.get(key)}: {val}")

    ...

sub_renewal_by_day('auto_purchase.log')